In [1]:
import time, cv2, sys, os, torch
from threading import Thread
from djitellopy import Tello
import openai

# load environment variables
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/prompts/")

# from vqa_question.v1 import prompt_template as vqa_prompt_template
from drone_explore_prompt import prompt_template as explore_template
from drone_next_action.v1 import prompt_template as drone_prompt_template

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Scripts/")

from data_collection_utils import take_individual_picture, stream_video, stream_frames, record_streamed_frames

print("Imported all modules")


/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Imported all modules


In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
drone = Tello()

In [ ]:
objective = "Find the person who is wearing a blue cap"

In [ ]:
drone.streamon()
drone.takeoff()

In [ ]:
def query_the_environment(drone, question):
    image = drone.get_frame_read().frame
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = torch.sigmoid(logits).argmax(-1).item()
    return str(model.config.id2label[idx])


In [ ]:
# Generate next action prompt
def next_action_prompt(prompt_template, objective, environment_context, previous_command=None):
    prompt = prompt_template.format(objective=objective, environment_context=environment_context, previous_command=previous_command)
    return prompt

In [2]:
def prompt(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.5,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=10,
        stop=["\n"]
    )
    return response.choices[0].text

In [ ]:
def explore(drone, use_gpt=False, prompt_template=explore_template):
    if use_gpt:
       prompt(prompt_template)
    else:
        import random as r
        # randomly generate a number between 45 and 180
        random_angle = r.randint(45, 180)
        # randomly generate a number between 5 and 20
        random_distance = r.randint(5, 20)
        actions = [(drone.move_left, random_distance), (drone.move_right, random_distance), (drone.rotate_clockwise, random_angle), (drone.rotate_counter_clockwise, random_angle), (drone.move_forward, random_distance), (drone.move_back, random_distance)]

        # sample 3 actions
        sampled_actions = r.sample(actions, 3)
        for action in sampled_actions:
            action[0](action[1])
            time.sleep(0.5)
